<a href="https://colab.research.google.com/github/naomifridman/Machine_learning_tutorials_and_utils/blob/master/google_colab_to_run_kaggle_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 4

import os
import time
import datetime
import json
import gc
from numba import jit

import numpy as np
import pandas as pd

from itertools import product

import lightgbm as lgb
import xgboost as xgb
import time
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


from IPython.display import HTML
import json

import matplotlib.pyplot as plt
%matplotlib inline



# Data load

In [0]:
DATA_IN_GOOGLE_CLOUD = False
DATA_ON_LOCAL_MACHINE = False
DATA_IN_KAGGLE = True
I_AM_IN_KAGGLE_KERNEL = False
PATH =''



## Load data from your computer
## Read csv  file from local
To read csv file  file from local machine, you need to upload it to your "workspace" and then call:
`df = pd.read_csv(<filename>)`



In [0]:
from google.colab import files

if DATA_ON_LOCAL_MACHINE:
    uploaded = files.upload()

## Load data from Google drive
* Create a drive for your project. Its not a must, but keep files in order.

## Use Google drive as storage
You can use your drive as disk, to save and load files. You need to:

### Allow usage of your drive.
In more accurate terms, mount a specific folder in your drive to a local disk.
* Run the following cell
* Open link
* Allow usage
* Copy key from window
* paste key bellow and press enter

## Filepath
Now you can accsess or create files, in the usual Python way,when:
`/content/gdrive/My Drive/`
Is the root path to your drive.
So use the following file path.
```
/content/gdrive/My Drive/<file path in your google drive tree>
```

In [0]:
from google.colab import drive

if DATA_IN_GOOGLE_CLOUD:
  PATH = '/content/gdrive/My Drive/' + 'Naya/kaggle/fraud/'

  drive.mount('/content/gdrive')

In [0]:
 if DATA_IN_GOOGLE_CLOUD:
    train_transaction = pd.read_csv(PATH + 'train_transaction.csv', index_col='TransactionID')
    test_transaction = pd.read_csv(PATH + 'test_transaction.csv', index_col='TransactionID')

    train_identity = pd.read_csv(PATH + 'train_identity.csv', index_col='TransactionID')
    test_identity = pd.read_csv(PATH + 'test_identity.csv', index_col='TransactionID')

    sample_submission = pd.read_csv(PATH + 'sample_submission.csv', index_col='TransactionID')

## Load data from Kagggle in Google colab
Thanks https://www.kaggle.com/aldrickpaul to for this answer, https://www.kaggle.com/general/51898
### Create an API token from your kaggle account.
* Choose `Edit Profile` in your 'My Profile' page.
* Use 'Create API Token' link to download kaggle.jason file.
* Open the jason file in any editor, and copy the keys to the bellow code:
* Copy the download command from the competition data page. Its in small leters near the download arrow. 

In [0]:
if DATA_IN_KAGGLE:
  os.environ['KAGGLE_USERNAME']='ripcurl' #xxxxxx
  os.environ['KAGGLE_KEY']='27c21759bd838377cc8c9f15c5bcbefd' #xxxxxx

  !kaggle competitions download -c ieee-fraud-detection # api copied from kaggle
  
  train_transaction = pd.read_csv('train_transaction.csv.zip', index_col='TransactionID')
  test_transaction = pd.read_csv('test_transaction.csv.zip', index_col='TransactionID')

  train_identity = pd.read_csv('train_identity.csv.zip', index_col='TransactionID')
  test_identity = pd.read_csv('test_identity.csv.zip', index_col='TransactionID')

  sample_submission = pd.read_csv('sample_submission.csv.zip', index_col='TransactionID')

 93% 49.0M/52.5M [00:00<00:00, 73.3MB/s]
100% 52.5M/52.5M [00:00<00:00, 118MB/s] 
  0% 0.00/3.02M [00:00<?, ?B/s]
100% 3.02M/3.02M [00:00<00:00, 99.1MB/s]
 80% 38.0M/47.3M [00:00<00:00, 47.4MB/s]
100% 47.3M/47.3M [00:00<00:00, 106MB/s] 
  0% 0.00/2.97M [00:00<?, ?B/s]
100% 2.97M/2.97M [00:00<00:00, 97.8MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 79.2MB/s]


In [0]:
NOT_FIRST_TIME=True

In [0]:
if DATA_IN_KAGGLE & NOT_FIRST_TIME:
  train_transaction = pd.read_csv(PATH+'train_transaction.csv.zip', index_col='TransactionID')
  test_transaction = pd.read_csv(PATH+'test_transaction.csv.zip', index_col='TransactionID')

  train_identity = pd.read_csv(PATH+'train_identity.csv.zip', index_col='TransactionID')
  test_identity = pd.read_csv(PATH+'test_identity.csv.zip', index_col='TransactionID')

  sample_submission = pd.read_csv(PATH+'sample_submission.csv.zip', index_col='TransactionID')

In [0]:
!ls 

sample_data		   test_identity.csv.zip     train_identity.csv.zip
sample_submission.csv.zip  test_transaction.csv.zip  train_transaction.csv.zip


## Load data from Kagggle when in kaggle kernel

In [0]:
if I_AM_IN_KAGGLE_KERNEL:
  PATH = '../input/'
  
  train_transaction = pd.read_csv(PATH+'train_transaction.csv', index_col='TransactionID')
  test_transaction = pd.read_csv(PATH+'test_transaction.csv', index_col='TransactionID')

  train_identity = pd.read_csv(PATH+'train_identity.csv', index_col='TransactionID')
  test_identity = pd.read_csv(PATH+'test_identity.csv', index_col='TransactionID')

  sample_submission = pd.read_csv(PATH+'sample_submission.csv', index_col='TransactionID')

## Basic EDA
Mostly taken from the foolowing notebooks:

* https://www.kaggle.com/artgor/eda-and-models

*

In [0]:
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)


In [0]:
#train=train.replace([np.inf, -np.inf], np.nan)
#test=test.replace([np.inf, -np.inf], np.nan)

## Delete cols and rows full of None

In [0]:
isna = train.isnull().sum(axis=0)



In [0]:
isna=isna.sort_values(ascending =False)
isna.iloc[0:10],len(train)

(id_24    585793
 id_25    585408
 id_07    585385
 id_08    585385
 id_21    585381
 id_26    585377
 id_22    585371
 id_23    585371
 id_27    585371
 dist2    552913
 dtype: int64, 590540)

In [0]:
isna.index, isna.loc[isna.index[4]]*100./len(train)*1.

(Index(['id_24', 'id_25', 'id_07', 'id_08', 'id_21', 'id_26', 'id_22', 'id_23',
        'id_27', 'dist2',
        ...
        'C5', 'C6', 'C7', 'C8', 'C10', 'C11', 'C12', 'C13', 'C14', 'isFraud'],
       dtype='object', length=433), 99.12639279303689)

In [0]:
col=isna.index[0]

In [0]:
cols = list(train.columns)


In [0]:
n_cols=0
for i in range(len(isna)):
  if (isna.loc[isna.index[i]]*100./len(train)*1. > 70):
    cols.remove(isna.index[i])
    n_cols+=1
    #print('deleting col: ', isna.index[i], 'num null: ', isna.loc[isna.index[i]])
    
print('deleted: ', n_cols, 'cols')    

deleting col:  id_24 num null:  585793
deleting col:  id_25 num null:  585408
deleting col:  id_07 num null:  585385
deleting col:  id_08 num null:  585385
deleting col:  id_21 num null:  585381
deleting col:  id_26 num null:  585377
deleting col:  id_22 num null:  585371
deleting col:  id_23 num null:  585371
deleting col:  id_27 num null:  585371
deleting col:  dist2 num null:  552913
deleting col:  D7 num null:  551623
deleting col:  id_18 num null:  545427
deleting col:  D13 num null:  528588
deleting col:  D14 num null:  528353
deleting col:  D12 num null:  525823
deleting col:  id_04 num null:  524216
deleting col:  id_03 num null:  524216
deleting col:  D6 num null:  517353
deleting col:  id_33 num null:  517251
deleting col:  id_10 num null:  515614
deleting col:  D8 num null:  515614
deleting col:  D9 num null:  515614
deleting col:  id_09 num null:  515614
deleting col:  id_30 num null:  512975
deleting col:  id_32 num null:  512954
deleting col:  id_34 num null:  512735
dele

In [0]:
test.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,170.0,87.0,1.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,NaN,418.0,203.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,299.0,87.0,4.0,NaN,aol.com,NaN,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,NaN,NaN,NaN,NaN,231.0,634.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,472.0,87.0,2635.0,NaN,hotmail.com,NaN,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,NaN,NaN,NaN,NaN,136.0,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,205.0,87.0,17.0,NaN,gmail.com,NaN,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,NaN,NaN,NaN,NaN,242.0,242.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,264.0,87.0,6.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,NaN,NaN,NaN,NaN,22.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
test[cols]

ValueError: ignored

In [0]:

train = train[cols]
test=test[cols]

In [0]:
isna = train.isna().sum(axis=1)
train['isna']=isna
isna=isna.sort_values(ascending =False)
isna.iloc[0:10],train.shape

(TransactionID
 3443104    146
 3461068    145
 3461362    145
 3461061    145
 3443122    145
 3509237    133
 3509200    133
 3509592    133
 3509348    133
 3509570    131
 dtype: int64, (590540, 225))

In [0]:
isna.plot(kind='hist')

In [0]:
#train=train.drop('isna',axis=1)

KeyError: ignored

In [0]:
train.loc[train['isFraud'] == 1, 'isna'] = 0
train = train.sort_values(by=['isna'], ascending=False)

train.head()

In [0]:
len(train.loc[train['isna'] >15]), len(train.loc[train['isna'] >=25])*100./len(train)

In [0]:
train= train[train['isna'] <= 25]

### Down sample

In [0]:
train=train.drop(train[train['isFraud']==0].sample(frac=.25).index)

In [0]:
#train = train.reset_index().drop(columns = 'TransactionID')


In [0]:
#test = test.reset_index().drop(columns = 'TransactionID')

In [0]:
# dummy
memo = ["card4", "M1", "M2", "M3", "P_emaildomain_bin", "P_emaildomain_suffix", 
        "M4", "M5", "M6", "M7", "M8", "M9", "ProductCD"]


train['memo']

In [0]:
train = pd.get_dummies(train, columns=memo, dummy_na=True)
test = pd.get_dummies(test, columns=memo, dummy_na=True)

In [0]:
# https://www.kaggle.com/jazivxt/safe-box

emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 'optonline.net': 'other',
          'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol',
          'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink', 'gmail.com': 'google', 'me.com': 'apple',
          'earthlink.net': 'other', 'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other',
          'hotmail.com': 'microsoft', 'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other',
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo', 'servicios-ta.com': 'other',
          'netzero.net': 'other', 'suddenlink.net': 'other', 'roadrunner.com': 'other', 'sc.rr.com': 'other',
          'live.fr': 'microsoft', 'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink',
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 'rocketmail.com': 'yahoo',
          'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 'ymail.com': 'yahoo', 'outlook.com': 'microsoft',
          'mail.com': 'other', 'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other',
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other',
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other', 'aol.com': 'aol',
          'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']

for c in ['P_emaildomain', 'R_emaildomain']:
    train[c + '_bin'] = train[c].map(emails)
    test[c + '_bin'] = test[c].map(emails)

    train[c + '_suffix'] = train[c].map(lambda x: str(x).split('.')[-1])
    test[c + '_suffix'] = test[c].map(lambda x: str(x).split('.')[-1])

    train[c + '_suffix'] = train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    test[c + '_suffix'] = test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')



In [0]:
def reduce_mem_usage(df):
   """ iterate through all the columns of a dataframe and modify the data type
       to reduce memory usage.
   """
   start_mem = df.memory_usage().sum() / 1024 ** 2
   print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

   for col in df.columns:
       col_type = df[col].dtype

       if col_type != object:
           c_min = df[col].min()
           c_max = df[col].max()
           if str(col_type)[:3] == 'int':
               if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                   df[col] = df[col].astype(np.int8)
               elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                   df[col] = df[col].astype(np.int16)
               elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                   df[col] = df[col].astype(np.int32)
               elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                   df[col] = df[col].astype(np.int64)
           else:
               if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                   df[col] = df[col].astype(np.float16)
               elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                   df[col] = df[col].astype(np.float32)
               else:
                   df[col] = df[col].astype(np.float64)
       else:
           df[col] = df[col].astype('category')

   end_mem = df.memory_usage().sum() / 1024 ** 2
   print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
   print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

   return df

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
gc.collect()

Memory usage of dataframe is 273.67 MB
Memory usage after optimization is: 271.92 MB
Decreased by 0.6%
Memory usage of dataframe is 480.99 MB
Memory usage after optimization is: 480.99 MB
Decreased by 0.0%
メモリ削減完了


0

In [0]:
# Label Encoding


for f in train.columns[1:]:
  if train[f].dtype == 'object' or test[f].dtype == 'object':
     lbl = LabelEncoder()
     lbl.fit(list(train[f].values) + list(test[f].values))
     train[f] = lbl.transform(list(train[f].values))
     test[f] = lbl.transform(list(test[f].values))


In [0]:
    START_DATE = "2017-12-01"

    
    
    memo = "TransactionDT"
    startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
    train["YMD"] = train[memo].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
    test["YMD"] = test[memo].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))

    memo = "YMD"
    print(train[memo].head())
    train[memo] = train[memo].astype(str)
    test[memo] = test[memo].astype(str)

    train["month"] = train[memo].map(lambda x: x.split("-")[1]).astype("int")
    # train["date"] = train[memo].map(lambda x: x.split("-")[2].split()[0]).astype("int")
    train["clock"] = train[memo].map(lambda x: x.split(":")[0].split()[1]).astype("int")

    test["month"] = test[memo].map(lambda x: x.split("-")[1]).astype("int")
    # test["date"] = test[memo].map(lambda x: x.split("-")[2].split()[0]).astype("int")
    test["clock"] = test[memo].map(lambda x: x.split(":")[0].split()[1]).astype("int")


    del train["YMD"], test["YMD"]

   


TransactionID
2987000   2017-12-02 00:00:00
2987001   2017-12-02 00:00:01
2987002   2017-12-02 00:01:09
2987003   2017-12-02 00:01:39
2987004   2017-12-02 00:01:46
Name: YMD, dtype: datetime64[ns]


In [0]:

del train_transaction, train_identity, test_transaction, test_identity
gc.collect()

635

In [0]:
# fill nul
train=train.replace([np.inf, -np.inf], np.nan)
train=train.fillna(0)
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,month,clock
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,4,13926,0.0,150.0,1,142.0,1,315.0,87.0,19.0,0.0,32,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,12,0
2987001,0,86401,29.0,4,2755,404.0,150.0,2,102.0,1,325.0,87.0,0.0,0.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,12,0
2987002,0,86469,59.0,4,4663,490.0,150.0,4,166.0,2,330.0,87.0,287.0,0.0,36,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,12,0
2987003,0,86499,50.0,4,18132,567.0,150.0,2,117.0,2,476.0,87.0,0.0,0.0,55,32,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,0.0,0.0,0.0,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,12,0
2987004,0,86506,50.0,1,4497,514.0,150.0,2,102.0,1,420.0,87.0,0.0,0.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,1,0.0,-480.0,1,1,166.0,0.0,542.0,144.0,0.0,0.0,3,0.0,0.0,0.0,2,1,1,7,162,32.0,268,3,1,0,1,1,1,1565,12,0


In [0]:

test=test.replace([np.inf, -np.inf], np.nan)
test=test.fillna(0)
test.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,...,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,month,clock
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3663549,18403224,31.95,4,10409,111.0,150.0,4,226.0,2,170.0,87.0,1.0,0.0,16,32,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,0.0,0.0,0.0,0.0,418.0,203.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,7,0
3663550,18403263,49.00,4,4272,111.0,150.0,4,226.0,2,299.0,87.0,4.0,0.0,2,32,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,0.0,0.0,0.0,0.0,231.0,634.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,7,0
3663551,18403310,171.00,4,4476,574.0,150.0,4,226.0,2,472.0,87.0,2635.0,0.0,19,32,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,0.0,0.0,0.0,0.0,136.0,136.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,7,0
3663552,18403310,284.95,4,10989,360.0,150.0,4,166.0,2,205.0,87.0,17.0,0.0,16,32,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,0.0,0.0,0.0,0.0,242.0,242.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,7,0
3663553,18403317,67.95,4,18018,452.0,150.0,2,117.0,2,264.0,87.0,6.0,0.0,16,32,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,22.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,7,0


## Save pre-proccessed data
you can save train-test data after EDA, then use it directly, next time.

You can save it to Google drive, your local machine or as kaggle data set.

In [0]:
SAVE_AFTER_EDA_TO_GOOGLE_DRIVE=False
SAVE_IN_KAGGLE=False
SAVE_AFTER_EDA_TO_LOCAL_MACHINE=False

## Save data to Google drive
If you have'nt mounted your google drive, do so by the instruction above.


In [0]:
if SAVE_AFTER_EDA_TO_GOOGLE_DRIVE:
    train.to_csv(PATH+'train.csv',index=False)
    test.to_csv(PATH+'test.csv',index=False)


## Save your data to your local machine

In [0]:
if SAVE_AFTER_EDA_TO_LOCAL_MACHINE:
  train.to_csv('train.csv',index=False)
  test.to_csv('test.csv',index=False)
  files.download('train.csv')
  files.download('test.csv')

## Save your data in Kaggle
Since kaggle issaving data only in 'commit', The only resonable way I found, is to add exit() after save , then run commit.

The run the notebook again and turn-off save.

In [0]:
if SAVE_IN_KAGGLE:
  train.to_csv('train.csv',index=False)
  test.to_csv('test.csv',index=False)
  exit()

In [0]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,month,clock
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,4,13926,0.0,150.0,1,142.0,1,315.0,87.0,19.0,0.0,32,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,12,0
2987001,0,86401,29.0,4,2755,404.0,150.0,2,102.0,1,325.0,87.0,0.0,0.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,12,0
2987002,0,86469,59.0,4,4663,490.0,150.0,4,166.0,2,330.0,87.0,287.0,0.0,36,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,12,0
2987003,0,86499,50.0,4,18132,567.0,150.0,2,117.0,2,476.0,87.0,0.0,0.0,55,32,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,0.0,0.0,0.0,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,3,2,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,2,2,2,86,136,0.0,461,4,2,2,2,2,2,2740,12,0
2987004,0,86506,50.0,1,4497,514.0,150.0,2,102.0,1,420.0,87.0,0.0,0.0,16,32,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,1,0.0,-480.0,1,1,166.0,0.0,542.0,144.0,0.0,0.0,3,0.0,0.0,0.0,2,1,1,7,162,32.0,268,3,1,0,1,1,1,1565,12,0


## Modeling

In [0]:
y_train = train['isFraud'].values
target = y_train
y_preds = np.zeros(sample_submission.shape[0])
y_oof = np.zeros(train.shape[0])


clf = xgb.XGBClassifier(
   n_estimators=500,
   max_depth=9,
   learning_rate=0.05,
   subsample=0.9,
   colsample_bytree=0.9,
   tree_method='gpu_hist'
)

X_tr, X_vl, y_tr, y_vl = train_test_split(train.drop(['isFraud'], axis=1).values,
                                          y_train, test_size=0.1,
                                          random_state=0)
clf.fit(X_tr, y_tr)
y_pred_train = clf.predict_proba(X_vl)[:, 1]

print('ROC AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))

y_preds = clf.predict_proba(np.array(test))[:,1]

# Predicting

In [0]:
sample_submission["isFraud"] = y_preds[:,1]
sample_submission.to_csv("submit.csv")
sample_submission.head()

,isFraud
TransactionID,
3663549,0.0
3663550,1.0
3663551,1.0
3663552,0.0
3663553,0.0


In [0]:
from google.colab import files
files.download('submit.csv')

## LGBM

In [0]:
n_fold = 5
folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=5)

In [0]:
params = {'num_leaves': 256,
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': 13,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9
         }
result_dict_lgb = train_model_classification(X=X, X_test=X_test, y=y, 
                                             params=params, folds=folds,
                                             model_type='lgb', eval_metric='auc',
                                             plot_feature_importance=True,
                                            verbose=500, 
                                             early_stopping_rounds=200,
                                             n_estimators=5000, 
                                             averaging='usual')

In [0]:
sub.head()

## Blending

In [0]:
# xgb_params = {'eta': 0.04,
#               'max_depth': 5,
#               'subsample': 0.85,
#               'objective': 'binary:logistic',
#               'eval_metric': 'auc',
#               'silent': True,
#               'nthread': -1,
#               'tree_method': 'gpu_hist'}
# result_dict_xgb = train_model_classification(X=X, X_test=X_test, y=y, params=xgb_params, folds=folds, model_type='xgb', eval_metric='auc', plot_feature_importance=False,
#                                                       verbose=500, early_stopping_rounds=200, n_estimators=5000, averaging='rank')

In [0]:
# test = test.sort_values('TransactionDT')
# test['prediction'] = result_dict_xgb['prediction']
# sub['isFraud'] = pd.merge(sub, test, on='TransactionID')['prediction']
# sub.to_csv('submission_xgb.csv', index=False)

In [0]:
# test = test.sort_values('TransactionDT')
# test['prediction'] = result_dict_lgb['prediction'] + result_dict_xgb['prediction']
# sub['isFraud'] = pd.merge(sub, test, on='TransactionID')['prediction']
# sub.to_csv('blend.csv', index=False)